Zach Tretter

June 2020

--------


# Step 2 - Target Data Clean Up

Workbook Scope : Process the CSV that has the historic fill time data.  Specifically add additional informative date columns (e.g. year, month, day of the year, etc) and also identify invalid dates (false negatives).

----------

#### Contents
* [Make Campground Nicknames](#Campground-Nicknames)
* [Clean up Fill Time Column](#Clean-up-Fill-Time-Column)
* [Create Additional Date Columns](#Create-Additional-Date-Columns)
* [Identify Logan Pass Open/Close Dates](#Logan-Pass-Dates)
* [Identify Invalid Dates](#Identify-Invalid-Dates)
* [Identify Wildfire Closures](#Wildfire-Dates)
* [Look for Anomalies](#Look-for-Anomalies)
* [Export to CSV](#Export-to-CSV)

In [1]:
import requests
from bs4 import BeautifulSoup

import time
from datetime import datetime as dt
from datetime import timedelta
import calendar

import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np

#### Import the raw csv

In [2]:
df = pd.read_csv('../data/01_filltimes_raw.csv')
df = df.drop(columns='Unnamed: 0')

In [3]:
# View the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39780 entries, 0 to 39779
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cg_name    39780 non-null  object
 1   date       39780 non-null  object
 2   did_fill   39780 non-null  int64 
 3   fill_time  8595 non-null   object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [4]:
df['available']=1

## Campground Nicknames

#### Shorter Names for Campgrounds

In [5]:
# Shorter single-word names for each campground

nicknames = {
    'Apgar': 'Apga',
    'Avalanche': 'Aval',
    'Bowman Lake': 'BoLa',
    'Cut Bank': 'CuBa',
    'Fish Creek': 'FiCr',
    'Kintla Lake': 'KiLa',
    'Logging Creek': 'LoCr',
    'Many Glacier': 'MaGl',
    'Quartz Creek': 'QuCr',
    'Rising Sun': 'RiSu',
    'Sprague Creek': 'SpCr',
    'St. Mary': 'StMa',
    'Two Medicine': 'TwoM'
}

df['nickname'] = df['cg_name'].map(nicknames)

In [6]:
df['key_name_date'] = df['date']+ '_' + df['nickname']

## Clean up Fill Time Column

#### Create a column for time as in 24 hour format

In [7]:
# The function to do it

def time_as_24hr_string(fill_time):
    if type(fill_time) == str:
        return dt.strftime(dt.strptime(fill_time, '%I:%M%p'), "%H:%M")
    else:
        return 0

In [8]:
# Create this column in the dataframe

df['time_24'] = df['fill_time'].apply(time_as_24hr_string)

#### Create a column for time as numerical hours since midnight

In [9]:
# The function to do it

def time_as_hours_from_midnight(fill_time):
    time_24 = time_as_24hr_string(fill_time)
    
    if time_24 != 0:
        hour_val, minute_val = time_24.split(":")
        secs_elapsed = timedelta(hours = int(hour_val), minutes = int(minute_val)).seconds
        return round(secs_elapsed / 3600,2)
    
    else:
        return int(0)

In [10]:
# Create this column in the dataframe

df['hours_after_midnight'] = df['fill_time'].apply(time_as_hours_from_midnight)

## Create Additional Date Columns

#### Make the 'date' column of type date

In [11]:
df['date'] = pd.to_datetime(df['date'])

#### Isolate specific date attributes to include

https://strftime.org/

* Year (e.g. '2017')
* Month as Number (e.g. '09')
* Month as Text (e.g. 'Sep')
* Day of Year (e.g. '173')
* Day of Month (e.g. '4')
* Weekday (e.g. 'Sat')

In [12]:
# Isolate Year
df['year'] = df['date'].apply(
    lambda x: dt.strftime(x, "%Y")).astype(int)

# Isolate Month as Number
df['month_num'] = df['date'].apply(
    lambda x: dt.strftime(x, "%m")).astype(int)

# Isolate Month as Text
df['month_text'] = df['date'].apply(
    lambda x: dt.strftime(x, "%b"))

# Isolate Day of Year
df['day_of_year'] = df['date'].apply(
    lambda x: dt.strftime(x, "%j")).astype(int)

# Isolate Day of Month
df['day_of_month'] = df['date'].apply(
    lambda x: dt.strftime(x, "%d")).astype(int)

# Isolate Day of Week as Text
df['weekday'] = df['date'].apply(
    lambda x: dt.strftime(x, "%a"))

# Key for Year_Month combo
df['key_year_month'] = df['year'].astype(str) + "_" + df['month_text']

#### Identify if High Season (July and August)

In [13]:
df['high_season']=((df['month_num']==7) | (df['month_num']==8)).astype(int)

#### Identify if Fri/Sat Night

In [14]:
df['weekend'] = ((df['weekday']=='Fri') | (df['weekday']=='Sat')).astype(int)

#### Identify if Date is day before, after, or of a Holiday

In [15]:
df['holiday']=0

In [16]:
# Independence Day
df.loc[(df['month_num']==7) & (
    (df['day_of_month']==3) |
    (df['day_of_month']==4) |
    (df['day_of_month']==5)),
      'holiday'] = 1

In [17]:
# Labor day is the first monday in September
labor_days = df.loc[(df['month_num']==9) 
                    & (df['day_of_month']<8) 
                    & (df['weekday']=='Mon')
                   ]['date'].unique()

# Memorial day is the last monday in May
memorial_days = df.loc[(df['month_num']==5) 
                       & (df['day_of_month'] > 24) 
                       & (df['weekday']=='Mon')
                      ]['date'].unique()

holiday_dates = []

for i in memorial_days:
    holiday_dates.append(i)
    holiday_dates.append(i - np.timedelta64(1,'D'))
    holiday_dates.append(i + np.timedelta64(1,'D'))
    
for i in labor_days:
    holiday_dates.append(i)
    holiday_dates.append(i - np.timedelta64(1,'D'))
    holiday_dates.append(i + np.timedelta64(1,'D'))

In [18]:
# Convert holiday to 1 for these dates
for day in holiday_dates:
    df.loc[df['date']==day,'holiday']==1

## Logan Pass Dates

#### Identify when Going to the Sun Road is Open

In [19]:
# Set every row as zero
df['logan_pass'] = 0

# Built in excel because its only 20 dates
# For that year, logan pass is open between these dates, as represent by a day of the year
logan_pass_dates = {
    2000:[148,290],
    2001:[158,295],
    2002:[179,301],
    2003:[150,301],
    2004:[151,292],
    2005:[142,304],
    2006:[174,296],
    2007:[182,259],
    2008:[184,294],
    2009:[177,291],
    2010:[175,291],
    2011:[194,261],
    2012:[171,289],
    2013:[172,266],
    2014:[183,265],
    2015:[170,291],
    2016:[168,286],
    2017:[179,274],
    2018:[174,274],
    2019:[174,274]
}

for YEAR in logan_pass_dates.keys():
    OPENDATE = logan_pass_dates[YEAR][0]
    CLOSEDATE = logan_pass_dates[YEAR][1]

    df.loc[(df['year'] == YEAR) & 
           ( (df['day_of_year'] >= OPENDATE)
            & (df['day_of_year'] <= CLOSEDATE) )
           , 'logan_pass'] = 1

## Identify Invalid Dates

In [20]:
# Examine a specific campground in detail

# specific_campground = 'Two Medicine'
# df.loc[(df['did_fill']==1) & 
#        (df['cg_name']==specific_campground)].sort_values('day_of_year')[['date','day_of_month']]

#### Earliest and Latest Date a Campground Ever Filled

In [21]:
max_min_fill = pd.DataFrame()

for campground in df['cg_name'].unique():
    max_min_fill = max_min_fill.append(
        df.loc[
            # Look at dates when the campground filled
            (df['did_fill']==1) & 
            
            # Look at a specific campground
            (df['cg_name']==campground),
            
            # Return these columns
            ['cg_name','date','day_of_year','did_fill','fill_time','month_text','day_of_month']
            
            # Sort by the the day of the year to get the earliest first and the latest last
            # .iloc returns the first and last element
        ].sort_values('day_of_year').iloc[[0,-1]]
    )

max_min_fill

,cg_name,date,day_of_year,did_fill,fill_time,month_text,day_of_month
4,Apgar,2000-05-05,126,1,9:25am,May,5
2586,Apgar,2016-09-16,260,1,7:06pm,Sep,16
5708,Avalanche,2017-06-17,168,1,12:44pm,Jun,17
5646,Avalanche,2016-09-16,260,1,6:17pm,Sep,16
8437,Bowman Lake,2015-05-23,143,1,5:27pm,May,23
8245,Bowman Lake,2013-09-14,257,1,4:42pm,Sep,14
10737,Cut Bank,2010-05-28,148,1,9:31am,May,28
12070,Cut Bank,2018-09-14,257,1,8:07pm,Sep,14
15178,Fish Creek,2019-06-01,152,1,5:47pm,Jun,1
14813,Fish Creek,2016-09-03,247,1,12:17pm,Sep,3


#### Built list of Open/Close Dates within Excel

In [22]:
cg_open_close = [
    ['Apgar',1,366],
    ['Avalanche',168,262],
    ['Bowman Lake',138,258],
    ['Cut Bank',148,269],
    ['Fish Creek',152,248],
    ['Kintla Lake',148,268],
    ['Logging Creek',180,268],
    ['Many Glacier',139,269],
    ['Quartz Creek',180,366],
    ['Rising Sun',154,256],
    ['Sprague Creek',126,266],
    ['St. Mary',1,366],
    ['Two Medicine',146,269]
]

# Set to 1
df['available'] = 1

#### Update the Dataframe

In [23]:
for entry in cg_open_close:
    
    CAMPGROUND = entry[0]
    OPENDATE = entry[1]
    CLOSEDATE = entry[2]

    df.loc[
        (df['cg_name'] == CAMPGROUND) &
        ( (df['day_of_year'] < OPENDATE) 
         | (df['day_of_year'] > CLOSEDATE) ),
         'available'] = 0

#### Verify no Fill Dates are Classified as Unvailable

In [24]:
pd.crosstab(df['available'],
            df['did_fill'])

did_fill,0,1
available,,
0,8895,0
1,22290,8595


## Wildfire Dates


#### Create Column where are all dates are zero

In [51]:
df['fire_closure'] = 0

#### 2018 Howe Ridge Fire

In [52]:
howe_ridge_fire =[
['Apgar',226,365],
['Avalanche',225,365],
['Bowman Lake',0,0],
['Cut Bank',0,0],
['Fish Creek',225,365],
['Kintla Lake',0,0],
['Logging Creek',0,0],
['Many Glacier',0,0],
['Quartz Creek',0,0],
['Rising Sun',0,0],
['Sprague Creek',225,365],
['St. Mary',0,0],
['Two Medicine',0,0],
]

for entry in howe_ridge_fire:
    
    CAMPGROUND = entry[0]
    START = entry[1]
    FINISH = entry[2]

    df.loc[
        ( (df['cg_name'] == CAMPGROUND) & (df['year']==2018) ) &
        ( (df['day_of_year'] >= START) 
         & (df['day_of_year'] <= FINISH) ),
         'fire_closure'] = 1

#### 2017 Sprague Fire

In [53]:
sprague_fire =[
['Apgar',246,365],
['Avalanche',246,365],
['Bowman Lake',0,0],
['Cut Bank',0,0],
['Fish Creek',246,365],
['Kintla Lake',0,0],
['Logging Creek',0,0],
['Many Glacier',0,0],
['Quartz Creek',0,0],
['Rising Sun',0,0],
['Sprague Creek',246,365],
['St. Mary',0,0],
['Two Medicine',0,0]

]

for entry in sprague_fire:
    
    CAMPGROUND = entry[0]
    START = entry[1]
    FINISH = entry[2]

    df.loc[
        ( (df['cg_name'] == CAMPGROUND) & (df['year']==201) ) &
        ( (df['day_of_year'] >= START) 
         | (df['day_of_year'] <= FINISH) ),
         'fire_closure'] = 1

#### 2015 Reynolds Fire

In [54]:
reynolds_fire=[
['Apgar',0,0],
['Avalanche',0,0],
['Bowman Lake',0,0],
['Cut Bank',0,0],
['Fish Creek',0,0],
['Kintla Lake',0,0],
['Logging Creek',0,0],
['Many Glacier',0,0],
['Quartz Creek',0,0],
['Rising Sun',203,365],
['Sprague Creek',0,0],
['St. Mary',203,211],
['Two Medicine',0,0],
]

for entry in reynolds_fire:
    
    CAMPGROUND = entry[0]
    START = entry[1]
    FINISH = entry[2]

    df.loc[
        ( (df['cg_name'] == CAMPGROUND) & (df['year']==2015) ) &
        ( (df['day_of_year'] >= START) 
         & (df['day_of_year'] < FINISH) ),
         'fire_closure'] = 1

### Reset Available to zero on these fire closure dates

In [55]:
df.loc[df['fire_closure']==1,'available'] = 0

#### Verify by Crosstab

In [58]:
pd.crosstab(df['fire_closure'],
            df['did_fill'])

did_fill,0,1
fire_closure,,
0,30911,8595
1,274,0


## Look for Anomalies

In [59]:
df[df['hours_after_midnight']>0].sort_values('hours_after_midnight')

,cg_name,date,did_fill,fill_time,available,nickname,key_name_date,time_24,hours_after_midnight,year,...,month_text,day_of_year,day_of_month,weekday,key_year_month,high_season,weekend,holiday,logan_pass,fire_closure
27322,Quartz Creek,2018-07-28,1,12:49am,1,QuCr,2018-07-28_QuCr,00:49,0.82,2018,...,Jul,209,28,Sat,2018_Jul,1,1,0,1,0
13851,Fish Creek,2010-07-21,1,5:26am,1,FiCr,2010-07-21_FiCr,05:26,5.43,2010,...,Jul,202,21,Wed,2010_Jul,1,0,0,1,0
13858,Fish Creek,2010-07-28,1,6:00am,1,FiCr,2010-07-28_FiCr,06:00,6.00,2010,...,Jul,209,28,Wed,2010_Jul,1,0,0,1,0
33442,Sprague Creek,2018-07-28,1,6:35am,1,SpCr,2018-07-28_SpCr,06:35,6.58,2018,...,Jul,209,28,Sat,2018_Jul,1,1,0,1,0
33577,Sprague Creek,2019-07-10,1,6:38am,1,SpCr,2019-07-10_SpCr,06:38,6.63,2019,...,Jul,191,10,Wed,2019_Jul,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14345,Fish Creek,2013-08-25,1,11:36pm,1,FiCr,2013-08-25_FiCr,23:36,23.60,2013,...,Aug,237,25,Sun,2013_Aug,1,0,0,1,0
33107,Sprague Creek,2016-06-29,1,11:37pm,1,SpCr,2016-06-29_SpCr,23:37,23.62,2016,...,Jun,181,29,Wed,2016_Jun,0,0,0,1,0
28371,Rising Sun,2005-07-06,1,11:47pm,1,RiSu,2005-07-06_RiSu,23:47,23.78,2005,...,Jul,187,6,Wed,2005_Jul,1,0,0,1,0
39252,Two Medicine,2016-07-24,1,11:52pm,1,TwoM,2016-07-24_TwoM,23:52,23.87,2016,...,Jul,206,24,Sun,2016_Jul,1,0,0,1,0


#### Manual Corrections

In [60]:
df.loc[(df['key_name_date']=='2018-07-28_QuCr'),'hours_after_midnight'] = 12.82

## Export to CSV

In [63]:
df.drop(columns = ['fire_closure'],inplace= True)

In [64]:
df.to_csv('../data/02_filltimes_clean.csv')